## 1-4 링크에 있는 것을 한꺼번에 내려받기
* 모든 이미지, 페이지를 한꺼번에 내려받는 방법

### 이번 절에서 배울 내용
* 상대 경로를 절대 경로로 변경하는 방법
* 링크에 있는 것을 추출하기 위한 재귀 처리

### 한꺼번에 다운받는 데 필요한 처리 내용
* a 태그의 링크 대상이 상대 경로이고 링크 대상이 HTML인 경우 추가적인 처리를 해야함. 이런 링크를 재귀적으로 다운받아야 함.
* a 태그의 href 속성에 링크 대상이 "../img/hoge.png" 처럼 상대경로로 적혀 있는 경우 상대 경로를 절대경로로 변경해야 함.
* urllib.parse.urljoin()를 사용해서 함. 
* urljoin(base, path)로 사용

In [1]:
from urllib.parse import urljoin
base = "http://example.com/html/a.html"
print( urljoin(base, "b.html") )
print( urljoin(base, "sub/c.html") )
print( urljoin(base, "../index.html") )
print( urljoin(base, "../img/hoge.png") )
print( urljoin(base, "../css/hoge.css") )

http://example.com/html/b.html
http://example.com/html/sub/c.html
http://example.com/index.html
http://example.com/img/hoge.png
http://example.com/css/hoge.css


In [3]:
from urllib.parse import urljoin
base = "http://example.com/html/a.html"
print( urljoin(base, "/hoge.html") )
print( urljoin(base, "http://otherExample.com/wiki") )
print( urljoin(base, "//anotherExample.org/test") ) # //로 시작하면 base를 무시함.

http://example.com/hoge.html
http://otherExample.com/wiki
http://anotherExample.org/test


### 재귀적으로 HTML을 처리하는 방법
1. HTML을 분석한다.
1. 링크를 추출한다.
1. 각 링크 대상에 다음과 같은 처리를 한다.
1. 파일을 다운받는다.
1. 파일이 HTML이라면 재귀적으로 (10로 돌아가서 순서를 처음부터 실행한다.

#### 모든 페이지를 한꺼번에 다운받는 프로그램
* 웹에 있는 파이썬 문서 중에 library 폴더 아래에 있는 모든 것을 다운받아 본다.
* 프로그램을 실행하면 다음과 같이 사이트 내부의 파일 또는 HTML 등을 모두 다운받는다.

In [ ]:
# 파이썬 매뉴얼을 재귀적으로 다운받는 프로그램
# 모듈 읽어 들이기 --- (※1)
from bs4 import BeautifulSoup
from urllib.request import *
from urllib.parse import *
from os import makedirs
import os.path, time, re
# 이미 처리한 파일인지 확인하기 위한 변수 --- (※2)
proc_files = {}
# HTML 내부에 있는 링크를 추출하는 함수 --- (※3)
def enum_links(html, base):
    soup = BeautifulSoup(html, "html.parser")
    links = soup.select("link[rel='stylesheet']") # CSS
    links += soup.select("a[href]") # 링크
    result = []
    # href 속성을 추출하고, 링크를 절대 경로로 변환 --- (※4)
    for a in links:
        href = a.attrs['href']
        url = urljoin(base, href)
        result.append(url)
    return result
# 파일을 다운받고 저장하는 함수 --- (※5)
def download_file(url):
    o = urlparse(url)
    savepath = "./" + o.netloc + o.path
    if re.search(r"/$", savepath): # 폴더라면 index.html
        savepath += "index.html"
    savedir = os.path.dirname(savepath)
    # 모두 다운됐는지 확인
    if os.path.exists(savepath): return savepath
    # 다운받을 폴더 생성
    if not os.path.exists(savedir):
        print("mkdir=", savedir)
        makedirs(savedir)
    # 파일 다운받기 --- (※6)
    try:
        print("download=", url)
        urlretrieve(url, savepath)
        time.sleep(1) # 1초 휴식 --- (※7)
        return savepath
    except:
        print("다운 실패: ", url)
        return None        
# HTML을 분석하고 다운받는 함수 --- (※8)
def analyze_html(url, root_url):
    savepath = download_file(url)
    if savepath is None: return
    if savepath in proc_files: return # 이미 처리됐다면 실행하지 않음 --- (※9)
    proc_files[savepath] = True
    print("analyze_html=", url)
    # 링크 추출 --- (※10)
    html = open(savepath, "r", encoding="utf-8").read()
    links = enum_links(html, url)
    for link_url in links:
        # 링크가 루트 이외의 경로를 나타낸다면 무시 --- (※11)
        if link_url.find(root_url) != 0:
            if not re.search(r".css$", link_url): continue
        # HTML이라면
        if re.search(r".(html|htm)$", link_url):
            # 재귀적으로 HTML 파일 분석하기
            analyze_html(link_url, root_url)
            continue
        # 기타 파일
        download_file(link_url)
if __name__ == "__main__":
    # URL에 있는 모든 것 다운받기 --- (※12)
    url = "https://docs.python.org/3.5/library/"
    analyze_html(url, url)

1. 필요한 모든 모듈을 읽어들인다. 인터넷에서 데이터를 내려받기 위해 urllib.request, URL 분석을 위한 urllib.parse, 폴더 생성을 위한 os, 경로와 관련된 os.path, 슬립을 위한 time ,정규 표현식을 위한 re 모듈을 읽어들임.
1. 전역 변수 pro_files를 초기화한다. 이는 이미 분석한 HTML 파일인지 판별하기 위한 변수이다. HTML 링크 구조는 a.html에서 b.html로 이동하는 링크가 있으면 b.html에서 a.html로 이동하는 변수가 있을 수 있음. 이때 따로 처리를 하지 않으면 무한 루프에 빠져서 처리가 종료되지 않아버린다. 따라서 이러한 변수를 사용해 같은 HTML에 두 번 이상 처리를 반복하지 않게 만들어야 한다.
1. enum_links() 함수에서는 HTML을 분석하고, 링크를 추출한다. a 캐그로 링크, link 태그로 스타일시트의 경로를 찾는데, 두 가지 모두 BeautifulSoup의 select() 메서드를 사용했다. 
1. 링크 태그의 href 속성에 적혀 있는 URL을 추출하고, 절대 경로로 변환한다.
1. 인터넷에 있는 파일을 다운로드 받는다. URL을 기반으로 파일명을 결정하고, 필요하다면 폴더를 생성한다.
1. 실제 파일을 다운로드 한다. urlretrieve() 함수를 사용한다.
1. 일시적으로 처리를 중지하기 위해 time.sleep() 메서드를 사용. sleep() 메서드는 특정한 시간동안 처리를 멈추는 기능을 제공하는데, 이는 파일을 다운로드하는 웹 서버에 부하를 주지 않기 위해서임
1. analyze_html() 함수에서는 HTML 파일을 분석하고, 링크에 있는 것을 다운받는다.
1. 같은 파일에 처리를 반복하지 않게 확인한다. 2에서 선언했던 변수를 사용한다.
1. 링크를 추출하고 6에서 본 것 처럼 urlretrieve() 함수로 다운받고, 이미 다운받은 파일을 읽어 들이는 처리를 진행한다.
1. 어느 사이트를 다운로드할지 지정. `__name__`에는 모듈 이름이 들어오게 되는데, 모듈이 아닌 경우에 "__main__"이 들어온다. 따라서 이 스크립트를 직접적으로 실행한 경우에  `__name__` 에 main이 들어와서 처리를 진행하게 된다. 이는 프로그램에서 정의하고 있는 함수를 다른 프로그램에서도 사용할 수 있게 고려한 것이다.